# COMS 4995_002 Deep Learning Assignment 1
Due on Monday, Oct 9, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Conrad De Peuter, cld2167

Member 2: Daniel First, df2450

Member 3: Arjun Singh, asd2171

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [362]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        
        eps = .01
        self.parameters = {'layer_dimensions':layer_dimensions, 'drop_prob':drop_prob, 'reg_lambda':reg_lambda} 
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        # init parameters
        for l in range(1, self.num_layers):
            # xavier init
            self.parameters['W' + str(l)] = np.random.normal(size=(layer_dimensions[l], layer_dimensions[l-1]))  * np.sqrt(2)/np.sqrt(layer_dimensions[l-1])
            self.parameters['b' + str(l)]= np.ones((layer_dimensions[l], 1)) * eps
    
        # print all params:
        print(self.parameters.keys())

        
#     def gradientCheck(self, theta, x):
#         eps = 1e-7
#         J_pos = J(theta + eps, x)
#         J_neg = J(theta - eps, x)
#         numerical_deriv = (J_pos - J-neg)/(2*eps)
#         analytic_deriv = JDeriv(theta, x)
#         error = norm(numerical_deriv-analyic_deriv)/norm()
#         return error
        
    def affineForward(self, A_, W_, b_):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
#         print("forward, shape W", W_.shape)
#         print("forward, shape A", A_.shape)
#         print("forward, shape b", b_.shape)
        return np.dot(W_, A_) + b_, [A_, W_, b_]
        

    def activationForward(self, A_, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        
        if activation == "relu":
            return self.relu(A_)
        if activation == "softmax":
            return self.softmax(A_)
        
        return A_

    def softmax(self, x):
        # softmax
        
        e_x = np.exp(x - np.max(x,axis=0))
        return e_x / e_x.sum( axis=0)
    
    def relu(self, X):
        return np.maximum(0,X)
            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        
        # make a mask, binary mask
        # where that mask is 0, drop those activation units
        # where its one, scale them up by 1/(1-p)
        # in forward we need to apply this mask
        
        M = np.random.binomial([np.ones(A.shape)],1-prob)[0] 
        A = np.multiply(A * (1/(1-prob)), M)
        
        return A, M

    def forwardPropagation(self, X, training=False):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = {}
        A = X
        
        for l in range(1, self.num_layers):
            
            layer_cache = {}
            layer_cache["linear"] = [A,  self.parameters["W" + str(l)], self.parameters["b" + str(l)]]
            Z, cache_l = self.affineForward(A, self.parameters["W" + str(l)], self.parameters["b" + str(l)])
            
            if l != self.num_layers - 1:
                if self.drop_prob > 0 and training:
                    Z, dropout_mask = self.dropout(Z, self.drop_prob)
                    layer_cache['dropout'] = dropout_mask
                        
                cache_a = self.activationForward(Z)
                
            else:
                cache_a = Z
            layer_cache['activation'] = cache_a
            cache[str(l)] = layer_cache
            
            A = cache_a
        
        return A, cache
    
    def costFunction(self, AL, y, reg):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """

        # softmax
        
        AL = self.softmax(AL)
        n_samples = AL.shape[1]
        true_labels = one_hot(y)
 
        y_hat = AL[y, range(y.shape[0])]
    
        # cost with regularization
        cost = -np.sum(np.log(y_hat))/ AL.shape[1] + (0.5)*self.reg_lambda * (1/n_samples) \
            * sum([scipy.linalg.norm(self.parameters["W" + str(l)],ord='fro') for l in range(1, self.num_layers)])
        
        if self.reg_lambda > 0:
            # add regularization
            pass
        
        # gradient of cost, should be same shape as input
        # 
        #dAL = -np.divide(true_labels, AL) 
        dAL = AL - true_labels
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A, W, b = cache['linear']
        n_samples = A.shape[1]
        #output_last_layer = cache[str(layer-1)]['activation']

     
        dW = (1/A.shape[1]) * np.dot(dA_prev,A.T) + self.reg_lambda * (1/n_samples) * W
        db = (1/A.shape[1]) * np.sum(dA_prev, axis=1, keepdims=True)
        
        dA_prev = np.dot(W.T,dA_prev)
        return dA_prev, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        
        zl = cache['activation']
        
        # dA is derivative last layer, first time it will be cost
        return np.multiply(dA , self.relu_derivative(zl, dA))  
    
    def relu_derivative(self, cached_x, dx):
        """
        Inputs:
        dx: upstream derivative
        cached_x: input of relu

        Returns:
        dx: gradient with respect to cached_x
        """     
        return 1.0 * (cached_x > 0)

    def dropout_backward(self, dA, cache):
        # in backwards we need to apply that mask to the derivatives
        # so cache here contains the dropout mask you used in forward prop
        # but also need to increase the nonzero elements by 1/(1-p)
        mask = cache['dropout']
        p = self.drop_prob
        return np.multiply((1/(1-p)) * mask, dA)

    def backPropagation(self, dAL, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        dA_prev = dAL
        
        for l in range(self.num_layers-1, 0, -1):
            cached_l = cache[str(l)]
            
            if l != self.num_layers - 1:
                dA_prev = self.activationBackward(dA_prev, cached_l)
    
            # affine backwards
            if l != self.num_layers - 1 and self.drop_prob > 0:
                #call dropout_backward
                dA_prev = self.dropout_backward(dA_prev, cached_l)
          
                
            dA_prev, dW, db = self.affineBackward(dA_prev ,cached_l)
            gradients["dW" + str(l)] = dW
            gradients["db" + str(l)] = db
        
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            pass
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for l in range(1, self.num_layers):
            #self.parameters["W" + str(l)] = self.parameters["W" + str(l)] - alpha * gradients["dW" + str(l)]
            self.parameters["W" + str(l)] = self.parameters["W" + str(l)] * (1 - self.reg_lambda/self.parameters["W" + str(1)].shape[0]) - alpha * gradients["dW" + str(l)]
            self.parameters["b" + str(l)] = self.parameters["b" + str(l)] - alpha * gradients["db" + str(l)]
            
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        # get validation as last 10%
        validation_size = int(X.shape[1]*.1)
        orig_size = X.shape[1]
        validation_X = X[:,orig_size-validation_size:]
        validation_y = y[orig_size-validation_size:]
        
        # remove validation data from X and y
        X = X[:,0:orig_size-validation_size]
        y = y[0:orig_size-validation_size]

        print("X train shape:", X.shape)
        print("y train shape:", y.shape)
        print("X validation shape:", validation_X.shape)
        print("y validation shape:", validation_y.shape)
        
        
        for i in range(0, iters):
           
            batch_x, batch_y = self.get_batch(X, y, batch_size)
            
            # forward prop
            last_layer, cache = self.forwardPropagation(batch_x, training=True)
            
            # compute loss
            cost, cost_deriv = self.costFunction(last_layer, batch_y, self.reg_lambda)

            # compute gradients
            gradients = self.backPropagation(cost_deriv, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)
            # gradually decrease alpha
            if i % 1000 == 0 and i != 0:
                alpha /= 2
                
            if i % print_every == 0:
                
                print("\n************")
                print("iteration","iter", i)
                print("COST", cost)
                print("alpha", alpha)
                print("************")
                
                # check train and validation accuracies
                preds = np.argmax(last_layer, axis=0)
                print("train accuracy = ", np.mean(preds == batch_y))
                forward_validation = self.predict(validation_X)
                preds_y = np.argmax(forward_validation, axis=0)
                print("validation set accuracy = ", np.mean(preds_y == validation_y))
                
                # check gradients at each layer
                for d in range(self.num_layers-1, 0, -1):
                    print("abs gradients dW"+str(d),np.mean(np.abs(gradients["dW"+str(d)])))
                
    def predict(self, X_):
        """
        Make predictions for each sample
        """
        forward, _ = self.forwardPropagation(X_, training=False)
        return forward

    def get_batch(self, X_, y_, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        # are minibatches random
        #get random set
        #print("get batch shape", X_.shape, y_.shape)
        ix = np.random.choice(X_.shape[1], batch_size, replace=True)
        X_batch = X_[:,ix]
        y_batch = y_[ix]
        return X_batch, y_batch

In [248]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [315]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [5]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [354]:
layer_dimensions = [X_train.shape[0],600, 300, 10]  # .51
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=5000, alpha=.1, batch_size=500, print_every=100)

dict_keys(['layer_dimensions', 'drop_prob', 'reg_lambda', 'W1', 'b1', 'W2', 'b2', 'W3', 'b3'])
X train shape: (3072, 45000)
y train shape: (45000,)
X validation shape: (3072, 5000)
y validation shape: (5000,)

************
iteration iter 0
COST 2.5873200541
alpha 0.1
************
train accuracy =  0.098
validation set accuracy =  0.1024
abs gradients dW3 0.016134170471
abs gradients dW2 0.00219109788071
abs gradients dW1 0.00242685099353

************
iteration iter 100
COST 1.95784143817
alpha 0.1
************
train accuracy =  0.294
validation set accuracy =  0.2892
abs gradients dW3 0.00383405460868
abs gradients dW2 0.000381260290204
abs gradients dW1 0.000561180161722

************
iteration iter 200
COST 1.79012503476
alpha 0.1
************
train accuracy =  0.38
validation set accuracy =  0.3582
abs gradients dW3 0.00339828302537
abs gradients dW2 0.000316218437189
abs gradients dW1 0.000431062799106

************
iteration iter 300
COST 1.73945549702
alpha 0.1
************
trai

validation set accuracy =  0.534
abs gradients dW3 0.00389064987093
abs gradients dW2 0.000631468592244
abs gradients dW1 0.000696610519501

************
iteration iter 3400
COST 1.08041675953
alpha 0.0125
************
train accuracy =  0.63
validation set accuracy =  0.5288
abs gradients dW3 0.00383110303876
abs gradients dW2 0.000642098185997
abs gradients dW1 0.000677734738679

************
iteration iter 3500
COST 1.18833288044
alpha 0.0125
************
train accuracy =  0.568
validation set accuracy =  0.5314
abs gradients dW3 0.00428439758033
abs gradients dW2 0.000689430551934
abs gradients dW1 0.000705724141746

************
iteration iter 3600
COST 1.26844573246
alpha 0.0125
************
train accuracy =  0.578
validation set accuracy =  0.53
abs gradients dW3 0.00544293841959
abs gradients dW2 0.000789593689598
abs gradients dW1 0.00088454284359

************
iteration iter 3700
COST 1.24348781982
alpha 0.0125
************
train accuracy =  0.586
validation set accuracy =  0.

In [355]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [356]:
print(y_predicted.shape)

(10, 10000)


In [358]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y)


[[-0.1988789   1.11858944  2.11988173 ..., -3.12474474 -1.13748057
   0.31780317]
 [-0.79655927  3.68665674 -2.86376603 ..., -4.3517551  -1.70098712
  -3.15381592]
 [ 1.44800427 -1.20573951  1.19856407 ...,  2.30899246  1.70109274
   0.85499306]
 ..., 
 [-3.2121288  -2.67931781 -2.47841384 ...,  0.46941867 -0.78203116
   2.06401689]
 [-0.46716456  3.71992809  2.39222404 ..., -0.67337641 -1.85242764
  -1.6213182 ]
 [-2.82544364  4.21266144 -3.67101772 ..., -1.89651115 -2.33117679
  -1.5515482 ]]


## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=.2, reg_lambda=1)
NN2.train(X_train, y_train, iters=5000, alpha=0.1, batch_size=500, print_every=100)

dict_keys(['layer_dimensions', 'drop_prob', 'reg_lambda', 'W1', 'b1', 'W2', 'b2', 'W3', 'b3'])
X train shape: (3072, 45000)
y train shape: (45000,)
X validation shape: (3072, 5000)
y validation shape: (5000,)

************
iteration iter 0
COST 2.67997276597
alpha 0.1
************
train accuracy =  0.074
validation set accuracy =  0.1022
abs gradients dW3 0.0204716890704
abs gradients dW2 0.00245835611524
abs gradients dW1 0.00229682502129

************
iteration iter 100
COST 2.06842425503
alpha 0.1
************
train accuracy =  0.254
validation set accuracy =  0.2778
abs gradients dW3 0.00438574173925
abs gradients dW2 0.000444978194801
abs gradients dW1 0.000586556705992

************
iteration iter 200
COST 1.93616812784
alpha 0.1
************
train accuracy =  0.348
validation set accuracy =  0.3504
abs gradients dW3 0.00214343634898
abs gradients dW2 0.000266980347298
abs gradients dW1 0.000285579719155

************
iteration iter 300
COST 1.8678925641
alpha 0.1
************
tr

validation set accuracy =  0.3194
abs gradients dW3 0.00281986632475
abs gradients dW2 0.000229307065037
abs gradients dW1 0.000135774993162

************
iteration iter 3400
COST 2.00544314299
alpha 0.0125
************
train accuracy =  0.302
validation set accuracy =  0.3132
abs gradients dW3 0.00335251022274
abs gradients dW2 0.000267626580172
abs gradients dW1 0.000163689504716

************
iteration iter 3500
COST 2.03524033777
alpha 0.0125
************
train accuracy =  0.256
validation set accuracy =  0.3126
abs gradients dW3 0.00251711355617
abs gradients dW2 0.000224270233813
abs gradients dW1 0.000156334307443

************
iteration iter 3600
COST 2.0136574969
alpha 0.0125
************
train accuracy =  0.298
validation set accuracy =  0.3008
abs gradients dW3 0.00245917414063
abs gradients dW2 0.000196174840884
abs gradients dW1 9.42575558987e-05

************
iteration iter 3700
COST 2.02125784416
alpha 0.0125
************
train accuracy =  0.268
validation set accuracy =

In [369]:
y_predicted2 = NN2.predict(X_test)
save_predictions('ans2-uni.npy', y_predicted)

(10, 10000)

cifar10-hw1/test/0.png
cifar10-hw1/test/1.png
cifar10-hw1/test/10.png
cifar10-hw1/test/100.png
cifar10-hw1/test/1000.png
cifar10-hw1/test/1001.png
cifar10-hw1/test/1002.png
cifar10-hw1/test/1003.png
cifar10-hw1/test/1004.png
cifar10-hw1/test/1005.png
cifar10-hw1/test/1006.png
cifar10-hw1/test/1007.png
cifar10-hw1/test/1008.png
cifar10-hw1/test/1009.png
cifar10-hw1/test/101.png
cifar10-hw1/test/1010.png
cifar10-hw1/test/1011.png
cifar10-hw1/test/1012.png
cifar10-hw1/test/1013.png
cifar10-hw1/test/1014.png
cifar10-hw1/test/1015.png
cifar10-hw1/test/1016.png
cifar10-hw1/test/1017.png
cifar10-hw1/test/1018.png
cifar10-hw1/test/1019.png
cifar10-hw1/test/102.png
cifar10-hw1/test/1020.png
cifar10-hw1/test/1021.png
cifar10-hw1/test/1022.png
cifar10-hw1/test/1023.png
cifar10-hw1/test/1024.png
cifar10-hw1/test/1025.png
cifar10-hw1/test/1026.png
cifar10-hw1/test/1027.png
cifar10-hw1/test/1028.png
cifar10-hw1/test/1029.png
cifar10-hw1/test/103.png
cifar10-hw1/test/1030.png
cifar10-hw1/test/1031.pn

cifar10-hw1/test/147.png
cifar10-hw1/test/1470.png
cifar10-hw1/test/1471.png
cifar10-hw1/test/1472.png
cifar10-hw1/test/1473.png
cifar10-hw1/test/1474.png
cifar10-hw1/test/1475.png
cifar10-hw1/test/1476.png
cifar10-hw1/test/1477.png
cifar10-hw1/test/1478.png
cifar10-hw1/test/1479.png
cifar10-hw1/test/148.png
cifar10-hw1/test/1480.png
cifar10-hw1/test/1481.png
cifar10-hw1/test/1482.png
cifar10-hw1/test/1483.png
cifar10-hw1/test/1484.png
cifar10-hw1/test/1485.png
cifar10-hw1/test/1486.png
cifar10-hw1/test/1487.png
cifar10-hw1/test/1488.png
cifar10-hw1/test/1489.png
cifar10-hw1/test/149.png
cifar10-hw1/test/1490.png
cifar10-hw1/test/1491.png
cifar10-hw1/test/1492.png
cifar10-hw1/test/1493.png
cifar10-hw1/test/1494.png
cifar10-hw1/test/1495.png
cifar10-hw1/test/1496.png
cifar10-hw1/test/1497.png
cifar10-hw1/test/1498.png
cifar10-hw1/test/1499.png
cifar10-hw1/test/15.png
cifar10-hw1/test/150.png
cifar10-hw1/test/1500.png
cifar10-hw1/test/1501.png
cifar10-hw1/test/1502.png
cifar10-hw1/test/1

cifar10-hw1/test/1974.png
cifar10-hw1/test/1975.png
cifar10-hw1/test/1976.png
cifar10-hw1/test/1977.png
cifar10-hw1/test/1978.png
cifar10-hw1/test/1979.png
cifar10-hw1/test/198.png
cifar10-hw1/test/1980.png
cifar10-hw1/test/1981.png
cifar10-hw1/test/1982.png
cifar10-hw1/test/1983.png
cifar10-hw1/test/1984.png
cifar10-hw1/test/1985.png
cifar10-hw1/test/1986.png
cifar10-hw1/test/1987.png
cifar10-hw1/test/1988.png
cifar10-hw1/test/1989.png
cifar10-hw1/test/199.png
cifar10-hw1/test/1990.png
cifar10-hw1/test/1991.png
cifar10-hw1/test/1992.png
cifar10-hw1/test/1993.png
cifar10-hw1/test/1994.png
cifar10-hw1/test/1995.png
cifar10-hw1/test/1996.png
cifar10-hw1/test/1997.png
cifar10-hw1/test/1998.png
cifar10-hw1/test/1999.png
cifar10-hw1/test/2.png
cifar10-hw1/test/20.png
cifar10-hw1/test/200.png
cifar10-hw1/test/2000.png
cifar10-hw1/test/2001.png
cifar10-hw1/test/2002.png
cifar10-hw1/test/2003.png
cifar10-hw1/test/2004.png
cifar10-hw1/test/2005.png
cifar10-hw1/test/2006.png
cifar10-hw1/test/200

cifar10-hw1/test/2458.png
cifar10-hw1/test/2459.png
cifar10-hw1/test/246.png
cifar10-hw1/test/2460.png
cifar10-hw1/test/2461.png
cifar10-hw1/test/2462.png
cifar10-hw1/test/2463.png
cifar10-hw1/test/2464.png
cifar10-hw1/test/2465.png
cifar10-hw1/test/2466.png
cifar10-hw1/test/2467.png
cifar10-hw1/test/2468.png
cifar10-hw1/test/2469.png
cifar10-hw1/test/247.png
cifar10-hw1/test/2470.png
cifar10-hw1/test/2471.png
cifar10-hw1/test/2472.png
cifar10-hw1/test/2473.png
cifar10-hw1/test/2474.png
cifar10-hw1/test/2475.png
cifar10-hw1/test/2476.png
cifar10-hw1/test/2477.png
cifar10-hw1/test/2478.png
cifar10-hw1/test/2479.png
cifar10-hw1/test/248.png
cifar10-hw1/test/2480.png
cifar10-hw1/test/2481.png
cifar10-hw1/test/2482.png
cifar10-hw1/test/2483.png
cifar10-hw1/test/2484.png
cifar10-hw1/test/2485.png
cifar10-hw1/test/2486.png
cifar10-hw1/test/2487.png
cifar10-hw1/test/2488.png
cifar10-hw1/test/2489.png
cifar10-hw1/test/249.png
cifar10-hw1/test/2490.png
cifar10-hw1/test/2491.png
cifar10-hw1/test

cifar10-hw1/test/301.png
cifar10-hw1/test/3010.png
cifar10-hw1/test/3011.png
cifar10-hw1/test/3012.png
cifar10-hw1/test/3013.png
cifar10-hw1/test/3014.png
cifar10-hw1/test/3015.png
cifar10-hw1/test/3016.png
cifar10-hw1/test/3017.png
cifar10-hw1/test/3018.png
cifar10-hw1/test/3019.png
cifar10-hw1/test/302.png
cifar10-hw1/test/3020.png
cifar10-hw1/test/3021.png
cifar10-hw1/test/3022.png
cifar10-hw1/test/3023.png
cifar10-hw1/test/3024.png
cifar10-hw1/test/3025.png
cifar10-hw1/test/3026.png
cifar10-hw1/test/3027.png
cifar10-hw1/test/3028.png
cifar10-hw1/test/3029.png
cifar10-hw1/test/303.png
cifar10-hw1/test/3030.png
cifar10-hw1/test/3031.png
cifar10-hw1/test/3032.png
cifar10-hw1/test/3033.png
cifar10-hw1/test/3034.png
cifar10-hw1/test/3035.png
cifar10-hw1/test/3036.png
cifar10-hw1/test/3037.png
cifar10-hw1/test/3038.png
cifar10-hw1/test/3039.png
cifar10-hw1/test/304.png
cifar10-hw1/test/3040.png
cifar10-hw1/test/3041.png
cifar10-hw1/test/3042.png
cifar10-hw1/test/3043.png
cifar10-hw1/test

...

cifar10-hw1/test/856.png
cifar10-hw1/test/8560.png
cifar10-hw1/test/8561.png
cifar10-hw1/test/8562.png
cifar10-hw1/test/8563.png
cifar10-hw1/test/8564.png
cifar10-hw1/test/8565.png
cifar10-hw1/test/8566.png
cifar10-hw1/test/8567.png
cifar10-hw1/test/8568.png
cifar10-hw1/test/8569.png
cifar10-hw1/test/857.png
cifar10-hw1/test/8570.png
cifar10-hw1/test/8571.png
cifar10-hw1/test/8572.png
cifar10-hw1/test/8573.png
cifar10-hw1/test/8574.png
cifar10-hw1/test/8575.png
cifar10-hw1/test/8576.png
cifar10-hw1/test/8577.png
cifar10-hw1/test/8578.png
cifar10-hw1/test/8579.png
cifar10-hw1/test/858.png
cifar10-hw1/test/8580.png
cifar10-hw1/test/8581.png
cifar10-hw1/test/8582.png
cifar10-hw1/test/8583.png
cifar10-hw1/test/8584.png
cifar10-hw1/test/8585.png
cifar10-hw1/test/8586.png
cifar10-hw1/test/8587.png
cifar10-hw1/test/8588.png
cifar10-hw1/test/8589.png
cifar10-hw1/test/859.png
cifar10-hw1/test/8590.png
cifar10-hw1/test/8591.png
cifar10-hw1/test/8592.png
cifar10-hw1/test/8593.png
cifar10-hw1/test

cifar10-hw1/test/8873.png
cifar10-hw1/test/8874.png
cifar10-hw1/test/8875.png
cifar10-hw1/test/8876.png
cifar10-hw1/test/8877.png
cifar10-hw1/test/8878.png
cifar10-hw1/test/8879.png
cifar10-hw1/test/888.png
cifar10-hw1/test/8880.png
cifar10-hw1/test/8881.png
cifar10-hw1/test/8882.png
cifar10-hw1/test/8883.png
cifar10-hw1/test/8884.png
cifar10-hw1/test/8885.png
cifar10-hw1/test/8886.png
cifar10-hw1/test/8887.png
cifar10-hw1/test/8888.png
cifar10-hw1/test/8889.png
cifar10-hw1/test/889.png
cifar10-hw1/test/8890.png
cifar10-hw1/test/8891.png
cifar10-hw1/test/8892.png
cifar10-hw1/test/8893.png
cifar10-hw1/test/8894.png
cifar10-hw1/test/8895.png
cifar10-hw1/test/8896.png
cifar10-hw1/test/8897.png
cifar10-hw1/test/8898.png
cifar10-hw1/test/8899.png
cifar10-hw1/test/89.png
cifar10-hw1/test/890.png
cifar10-hw1/test/8900.png
cifar10-hw1/test/8901.png
cifar10-hw1/test/8902.png
cifar10-hw1/test/8903.png
cifar10-hw1/test/8904.png
cifar10-hw1/test/8905.png
cifar10-hw1/test/8906.png
cifar10-hw1/test/

cifar10-hw1/test/9179.png
cifar10-hw1/test/918.png
cifar10-hw1/test/9180.png
cifar10-hw1/test/9181.png
cifar10-hw1/test/9182.png
cifar10-hw1/test/9183.png
cifar10-hw1/test/9184.png
cifar10-hw1/test/9185.png
cifar10-hw1/test/9186.png
cifar10-hw1/test/9187.png
cifar10-hw1/test/9188.png
cifar10-hw1/test/9189.png
cifar10-hw1/test/919.png
cifar10-hw1/test/9190.png
cifar10-hw1/test/9191.png
cifar10-hw1/test/9192.png
cifar10-hw1/test/9193.png
cifar10-hw1/test/9194.png
cifar10-hw1/test/9195.png
cifar10-hw1/test/9196.png
cifar10-hw1/test/9197.png
cifar10-hw1/test/9198.png
cifar10-hw1/test/9199.png
cifar10-hw1/test/92.png
cifar10-hw1/test/920.png
cifar10-hw1/test/9200.png
cifar10-hw1/test/9201.png
cifar10-hw1/test/9202.png
cifar10-hw1/test/9203.png
cifar10-hw1/test/9204.png
cifar10-hw1/test/9205.png
cifar10-hw1/test/9206.png
cifar10-hw1/test/9207.png
cifar10-hw1/test/9208.png
cifar10-hw1/test/9209.png
cifar10-hw1/test/921.png
cifar10-hw1/test/9210.png
cifar10-hw1/test/9211.png
cifar10-hw1/test/9

cifar10-hw1/test/9471.png
cifar10-hw1/test/9472.png
cifar10-hw1/test/9473.png
cifar10-hw1/test/9474.png
cifar10-hw1/test/9475.png
cifar10-hw1/test/9476.png
cifar10-hw1/test/9477.png
cifar10-hw1/test/9478.png
cifar10-hw1/test/9479.png
cifar10-hw1/test/948.png
cifar10-hw1/test/9480.png
cifar10-hw1/test/9481.png
cifar10-hw1/test/9482.png
cifar10-hw1/test/9483.png
cifar10-hw1/test/9484.png
cifar10-hw1/test/9485.png
cifar10-hw1/test/9486.png
cifar10-hw1/test/9487.png
cifar10-hw1/test/9488.png
cifar10-hw1/test/9489.png
cifar10-hw1/test/949.png
cifar10-hw1/test/9490.png
cifar10-hw1/test/9491.png
cifar10-hw1/test/9492.png
cifar10-hw1/test/9493.png
cifar10-hw1/test/9494.png
cifar10-hw1/test/9495.png
cifar10-hw1/test/9496.png
cifar10-hw1/test/9497.png
cifar10-hw1/test/9498.png
cifar10-hw1/test/9499.png
cifar10-hw1/test/95.png
cifar10-hw1/test/950.png
cifar10-hw1/test/9500.png
cifar10-hw1/test/9501.png
cifar10-hw1/test/9502.png
cifar10-hw1/test/9503.png
cifar10-hw1/test/9504.png
cifar10-hw1/test/

cifar10-hw1/test/9763.png
cifar10-hw1/test/9764.png
cifar10-hw1/test/9765.png
cifar10-hw1/test/9766.png
cifar10-hw1/test/9767.png
cifar10-hw1/test/9768.png
cifar10-hw1/test/9769.png
cifar10-hw1/test/977.png
cifar10-hw1/test/9770.png
cifar10-hw1/test/9771.png
cifar10-hw1/test/9772.png
cifar10-hw1/test/9773.png
cifar10-hw1/test/9774.png
cifar10-hw1/test/9775.png
cifar10-hw1/test/9776.png
cifar10-hw1/test/9777.png
cifar10-hw1/test/9778.png
cifar10-hw1/test/9779.png
cifar10-hw1/test/978.png
cifar10-hw1/test/9780.png
cifar10-hw1/test/9781.png
cifar10-hw1/test/9782.png
cifar10-hw1/test/9783.png
cifar10-hw1/test/9784.png
cifar10-hw1/test/9785.png
cifar10-hw1/test/9786.png
cifar10-hw1/test/9787.png
cifar10-hw1/test/9788.png
cifar10-hw1/test/9789.png
cifar10-hw1/test/979.png
cifar10-hw1/test/9790.png
cifar10-hw1/test/9791.png
cifar10-hw1/test/9792.png
cifar10-hw1/test/9793.png
cifar10-hw1/test/9794.png
cifar10-hw1/test/9795.png
cifar10-hw1/test/9796.png
cifar10-hw1/test/9797.png
cifar10-hw1/tes

In [ ]:
T = one_hot(test)
T

In [25]:
np.random.randint(10)

3

## Part 3, ensemble learning

Use a few networks with different arcitechtures to predict the photo class